In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import os
import tarfile
from six.moves import urllib

import pickle

In [2]:
#!pip install minisom

In [3]:
dataset = pd.read_csv('data/palavras-selecionadas.csv', index_col=0 )


In [4]:
dataset.head()

,absolvicao,administracao,anulacao,apreensao,associacao,aumento,autoral,banco,cargo,carreira,...,servico,servidor,social,taxa,trafico,tutela,veiculo,vicios,violacao,artigos
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"['art 1', 'art 487']"
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,['art 157']
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,['art 3']
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"['art 942', 'art 85', 'art 98']"
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,['art 487']


In [5]:
X = dataset.iloc[:, :-1].values


In [6]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 2., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
X = sc.fit_transform(X)

In [8]:
# Training the SOM
from minisom import MiniSom
som = MiniSom(x = 20, y = 20, input_len = dataset.shape[1]-1, sigma = 1.0, learning_rate = 0.5, random_seed = 4214)
som.random_weights_init(X)
som.train_random(data = X, num_iteration = 5000)

In [9]:
def palavras_contidas(dado):
    palavras = []
    for i in range(len(dado.tolist()[:-1])):
        if dado.tolist()[i] > 0:
            palavras.append(dado.index[i])
    return palavras

def exibe_palavas(dado):
    palavras = palavras_contidas(dado)
    for palavra in palavras:
        print(palavra)
    print('\nNúmero de palavras: {}'.format(len(palavras)))    
    
def acha_dado_com_palavra(palavra):
    numero_documentos_com_essa_palavra = 0
    lista = dataset[palavra].tolist()[:-1]
    for i in range(len(lista)):
        if lista[i] > 0:
            numero_documentos_com_essa_palavra += 1
            print(i)
    print('\nNúmero de documentos com essa palavra: {}'.format(numero_documentos_com_essa_palavra))
    
def palavras_em_comum(dado1, dado2):
    conta_palavras = 0
    palavras1 = palavras_contidas(dado1)
    palavras2 = palavras_contidas(dado2)
    for palavra in palavras1:
        if palavra in palavras2:
            print(palavra)
            conta_palavras += 1
    print('\nNúmero de palavras: {}'.format(conta_palavras))   

# Criação da matriz de posição de todos os elementos

In [10]:
arrX = list()
arrY = list()

for i in dataset.values:
    vencedor = som.winner(i[:-1])
    arrX.append(vencedor[0])
    arrY.append(vencedor[1])
    
    

In [11]:
# dataset['posicaoX'] = arrX
# dataset['posicaoY'] = arrY

In [29]:
findX = 0
findY = 0

array_mesma_posicao = list()


for i in range(len(dataset.values)):
    if arrX[i] == findX and arrY[i] == findY:
        array_mesma_posicao.append(i)

In [30]:
array_mesma_posicao

[446, 551, 867, 1017, 1320]

In [31]:
exibe_palavas(dataset.iloc[326])

servico

Número de palavras: 1


# Exibição das palavras do grupo

In [32]:
for i in array_mesma_posicao:
    print("\n\n------ Dado: {} ------".format(i))
    exibe_palavas(dataset.iloc[i])



------ Dado: 446 ------
absolvicao
concurso
crime
flagrante
policial
prisao
probatorio
roubo

Número de palavras: 8


------ Dado: 551 ------
constrangimento
custodia
flagrante
habeas
posse
prisao
trafico

Número de palavras: 7


------ Dado: 867 ------
absolvicao
apreensao
flagrante
policial
posse
prisao
probatorio
trafico

Número de palavras: 8


------ Dado: 1017 ------
crime
drogas
flagrante
habeas
militar
policial
prisao
probatorio
trafico

Número de palavras: 9


------ Dado: 1320 ------
constrangimento
crime
custodia
flagrante
habeas
prisao
roubo

Número de palavras: 7


# Artigos do Grupo

In [16]:
for i in array_mesma_posicao:
    print(dataset.iloc[i][-1])

['art 37']
['art 5']
['art 37']


# Contar artigos

In [17]:
lista_artigos = list()
conta_artigos = {}
for i in array_mesma_posicao:
    # transformar string em lista     
    art_string = dataset.iloc[i][-1]
    art_string = art_string.replace('[', '').replace(']', '').replace("'", '')
    art_list = art_string.split(',')
    
    for art in art_list:
        art = art.strip()
        if art not in lista_artigos:
            lista_artigos.append(art)
            conta_artigos[art] = 1
        else:
            conta_artigos[art] += 1
            
        

In [18]:
conta_artigos

{'art 37': 2, 'art 5': 1}

# Criando o dataframe de avaliação

In [19]:
df_avalia = pd.DataFrame(data = arrX, columns = ["posX"])
df_avalia['posY'] = arrY
df_avalia['artigos'] = dataset[['artigos']]

In [20]:
df_avalia.head()


,posX,posY,artigos
0,9,5,"['art 1', 'art 487']"
1,10,12,['art 157']
2,7,0,['art 3']
3,1,19,"['art 942', 'art 85', 'art 98']"
4,12,15,['art 487']


# Função de encontrar dados na mesma posição

In [21]:
def encontra_mesma_posicao(df_avalia, findX, findY):
    array_mesma_posicao = list()
    for i in range(len(df_avalia.values)):
        if df_avalia.values[i][0] == findX and df_avalia.values[i][1] == findY:
            array_mesma_posicao.append(i)
    return array_mesma_posicao

In [22]:
encontra_mesma_posicao(df_avalia, 3, 4)

[]

# Função de contar artigos

In [23]:
def conta_artigo(df_avalia, findX, findY):
    lista_artigos = list()
    conta_artigos = {}
    for i in encontra_mesma_posicao(df_avalia, findX, findY):
        # transformar string em lista     
        art_string = df_avalia.iloc[i][-1]
        art_string = art_string.replace('[', '').replace(']', '').replace("'", '')
        art_list = art_string.split(',')

        for art in art_list:
            art = art.strip()
            if art not in lista_artigos:
                lista_artigos.append(art)
                conta_artigos[art] = 1
            else:
                conta_artigos[art] += 1
    return conta_artigos

# Função de transformação de palavras em vetor

In [24]:
def vetorizar_palavras(lista_palavras):
    array = 60 * [0]
    for palavra in lista_palavras:
        if palavra in list(dataset.columns):
            posicao = list(dataset.columns).index(palavra)
            array[posicao] += 1
    return array

# Exportando o modelo(não precisa)

In [25]:
with open('model_minisom', 'wb') as f:
    pickle.dump(som, f)

# exemplo de importação (não precisa)

In [26]:
with open('model_minisom', 'rb') as f:
    model = pickle.load(f)

In [27]:
model.winner(dataset.iloc[2].values[:-1])

(7, 0)

In [28]:
type(model)

minisom.MiniSom

In [29]:
df_avalia.to_csv('data/df_avalia.csv')

# Flask

In [30]:
from flask import Flask
from flask import request
from flask import jsonify
from flask import Response
from flask_cors import CORS 

In [31]:
# !pip3 install -U flask-cors

In [32]:
app = Flask(__name__)

CORS(app)
cors = CORS(app, resources={r"/artigos/*": {"origins": "*"}})

@app.route('/artigos', methods=['POST'])
def index():
    palavras = request.json['palavras']
    print(palavras)
    array = vetorizar_palavras(palavras)
    posicao = som.winner(array)
    print("pX: {}".format(posicao[0]))
    print("pY: {}".format(posicao[1]))
    
    artigos = conta_artigo(df_avalia, posicao[0], posicao[1])

    # return jsonify(dictionary)
    return jsonify(artigos)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
